In [1]:
# attempt to reproduce xpriv derivation from xpub and child private key as per https://bitcoinmagazine.com/technical/deterministic-wallets-advantages-flaw-1385450276


# Library code taken from https://github.com/vbuterin/pybitcointools/tree/patch-1/cryptos

In [2]:
from pybctexcerpt import *

In [3]:
w = bip32_master_key('qweqweqweqweqwe')

In [4]:
print(w)

xprv9s21ZrQH143K2KhRQVuMqhz798mvW89J1aJMWEKNfZzv3BPAgBc4TH59K8ZcLt8RrNJvbUzA72A92Grm3MorG2FnFaoZ7B8SDTYXgUBALoi


In [5]:
wp = bip32_privtopub(w)
print(wp)

xpub661MyMwAqRbcEomtWXSNCqvqhAcQuas9NoDxJcizDuXtuyiKDivK15PdAPVkPwVXT9rFbjAnE9P3sLh6xnDawXF1uUXrruH1UvALHF89qdP


In [6]:
w0 = bip32_ckd(w,0)
print(w0)

xprv9uyTuGongdyZAMxZ2euUBbpsAdtE2nxFBmcQn89UT4ZyzrMg5TXD7azCnsnpH9Q7yrYgG7nVakE6BTxJUarLrDA28VxS3ZWDsgYWZUxtNiH


In [7]:
cracked = crack_bip32_privkey(wp,w0)
print(cracked)

xprv9s21ZrQH143K2KhRQVuMqhz798mvW89J1aJMWEKNfZzv3BPAgBc4TH59K8ZcLt8RrNJvbUzA72A92Grm3MorG2FnFaoZ7B8SDTYXgUBALoi


In [8]:
cracked == w

True